# 🔷 Glimpse3D - TripoSR 3D Reconstruction

**Fast single-image to 3D mesh reconstruction using TripoSR**

This notebook generates the **initial 3D mesh** from a single image in ~0.5 seconds.

## Pipeline Role
```
[This Notebook] → Mesh → Sample Points → Gaussian Splats → SyncDreamer → Refinement
```

## Requirements
- Google Colab with **T4 GPU** (free tier works!)
- ~6GB VRAM for default settings

---

## 1️⃣ Check GPU & Environment

In [ ]:
# Check environment
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Colab: {IN_COLAB}")

# Check GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2️⃣ Install Dependencies

In [ ]:
%%capture
# Install TripoSR dependencies
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install transformers accelerate huggingface_hub --quiet
!pip install omegaconf einops trimesh rembg[gpu] Pillow --quiet
!pip install xatlas plyfile --quiet

# Install torchmcubes for mesh extraction
!pip install git+https://github.com/tatsy/torchmcubes.git --quiet

print("✅ Dependencies installed!")

## 3️⃣ Clone TripoSR Repository

In [ ]:
import os
import sys

TRIPOSR_PATH = "/content/TripoSR"

if not os.path.exists(TRIPOSR_PATH):
    print("📥 Cloning TripoSR...")
    !git clone https://github.com/VAST-AI-Research/TripoSR.git {TRIPOSR_PATH}
else:
    print("✅ TripoSR already cloned")

# Add to path
if TRIPOSR_PATH not in sys.path:
    sys.path.insert(0, TRIPOSR_PATH)

os.chdir(TRIPOSR_PATH)
print(f"📂 Working directory: {os.getcwd()}")

## 4️⃣ Load TripoSR Model

In [ ]:
import torch
from tsr.system import TSR

# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model (downloads from HuggingFace automatically)
print("\n📥 Loading TripoSR model (first run downloads ~1GB)...")
model = TSR.from_pretrained(
    "stabilityai/TripoSR",
    config_name="config.yaml",
    weight_name="model.ckpt",
)

# Optimize for T4 GPU
model.renderer.set_chunk_size(8192)  # Balance speed/memory
model.to(device)
print("✅ Model loaded!")

## 5️⃣ Upload Your Image

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

# Upload image
print("📤 Upload an image (JPG/PNG):")
uploaded = files.upload()

# Get filename
IMAGE_PATH = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {IMAGE_PATH}")

# Display
img = Image.open(IMAGE_PATH)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')
plt.title("Input Image")
plt.show()

## 6️⃣ Preprocess Image (Remove Background)

In [ ]:
import rembg
import numpy as np
from PIL import Image
from tsr.utils import remove_background, resize_foreground

# Settings
REMOVE_BG = True  # Set to False if image already has transparent background
FOREGROUND_RATIO = 0.85  # How much of image the object should fill

# Load image
input_image = Image.open(IMAGE_PATH)

if REMOVE_BG:
    print("🔧 Removing background...")
    rembg_session = rembg.new_session()
    processed_image = remove_background(input_image, rembg_session)
    processed_image = resize_foreground(processed_image, FOREGROUND_RATIO)
    
    # Convert RGBA to RGB with gray background
    image_np = np.array(processed_image).astype(np.float32) / 255.0
    image_np = image_np[:, :, :3] * image_np[:, :, 3:4] + (1 - image_np[:, :, 3:4]) * 0.5
    processed_image = Image.fromarray((image_np * 255.0).astype(np.uint8))
else:
    processed_image = input_image.convert("RGB")

# Save processed
processed_image.save("/content/processed_input.png")

# Display
plt.figure(figsize=(6, 6))
plt.imshow(processed_image)
plt.axis('off')
plt.title("Processed Image")
plt.show()
print("✅ Preprocessing complete!")

## 7️⃣ Run TripoSR Inference

In [ ]:
import time

# Settings
MC_RESOLUTION = 256  # Marching cubes resolution (higher = more detailed)

print("🚀 Running TripoSR inference...")
start_time = time.time()

with torch.no_grad():
    # Generate scene codes
    scene_codes = model([processed_image], device=device)
    
    # Extract mesh with vertex colors
    meshes = model.extract_mesh(
        scene_codes, 
        has_vertex_color=True, 
        resolution=MC_RESOLUTION
    )

mesh = meshes[0]
elapsed = time.time() - start_time

print(f"\n✅ Mesh generated in {elapsed:.2f} seconds!")
print(f"   Vertices: {len(mesh.vertices):,}")
print(f"   Faces: {len(mesh.faces):,}")

## 8️⃣ Export Mesh (OBJ & PLY)

In [ ]:
import os

# Create output directory
OUTPUT_DIR = "/content/triposr_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Export OBJ (with vertex colors)
obj_path = f"{OUTPUT_DIR}/mesh.obj"
mesh.export(obj_path)
print(f"✅ Saved OBJ: {obj_path}")

# Export GLB (for web viewers)
glb_path = f"{OUTPUT_DIR}/mesh.glb"
mesh.export(glb_path)
print(f"✅ Saved GLB: {glb_path}")

# Export PLY (for Gaussian Splatting)
ply_path = f"{OUTPUT_DIR}/mesh.ply"
mesh.export(ply_path)
print(f"✅ Saved PLY: {ply_path}")

## 9️⃣ Convert to Gaussian Splat Format

In [ ]:
import numpy as np
from plyfile import PlyData, PlyElement

def mesh_to_gaussian_ply(mesh, output_path, num_samples=100000):
    """
    Convert trimesh mesh to Gaussian Splat PLY format.
    Samples points from mesh surface and initializes Gaussian parameters.
    """
    print(f"🔄 Sampling {num_samples:,} points from mesh...")
    
    # Sample points with colors from mesh
    points, face_indices = mesh.sample(num_samples, return_index=True)
    
    # Get vertex colors for sampled points
    if mesh.visual.vertex_colors is not None:
        # Interpolate colors from face vertices
        face_vertices = mesh.faces[face_indices]
        vertex_colors = mesh.visual.vertex_colors[:, :3] / 255.0
        
        # Simple: use average of face vertex colors
        colors = vertex_colors[face_vertices].mean(axis=1)
    else:
        colors = np.ones((num_samples, 3)) * 0.5  # Gray default
    
    num_points = len(points)
    
    # Initialize Gaussian parameters
    xyz = points.astype(np.float32)
    
    # SH DC coefficients (color)
    # Formula: RGB = 0.5 + C0 * SH_DC, where C0 = 0.28209
    C0 = 0.28209479177387814
    features_dc = ((colors - 0.5) / C0).astype(np.float32)
    
    # SH rest coefficients (15 * 3 = 45 for degree 3)
    features_rest = np.zeros((num_points, 45), dtype=np.float32)
    
    # Opacity (pre-activation, inverse sigmoid)
    # sigmoid(2.2) ≈ 0.9
    opacities = np.ones((num_points, 1), dtype=np.float32) * 2.2
    
    # Scales (pre-activation, log scale)
    # exp(-4.6) ≈ 0.01
    scales = np.ones((num_points, 3), dtype=np.float32) * (-4.6)
    
    # Rotations (quaternion: w, x, y, z)
    rotations = np.zeros((num_points, 4), dtype=np.float32)
    rotations[:, 0] = 1.0  # Identity rotation
    
    # Build PLY structure
    dtype_full = [
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
    ]
    
    # Add f_rest attributes
    for i in range(45):
        dtype_full.append((f'f_rest_{i}', 'f4'))
    
    dtype_full.extend([
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
    ])
    
    # Create structured array
    elements = np.zeros(num_points, dtype=dtype_full)
    
    elements['x'] = xyz[:, 0]
    elements['y'] = xyz[:, 1]
    elements['z'] = xyz[:, 2]
    elements['f_dc_0'] = features_dc[:, 0]
    elements['f_dc_1'] = features_dc[:, 1]
    elements['f_dc_2'] = features_dc[:, 2]
    
    for i in range(45):
        elements[f'f_rest_{i}'] = features_rest[:, i]
    
    elements['opacity'] = opacities[:, 0]
    elements['scale_0'] = scales[:, 0]
    elements['scale_1'] = scales[:, 1]
    elements['scale_2'] = scales[:, 2]
    elements['rot_0'] = rotations[:, 0]
    elements['rot_1'] = rotations[:, 1]
    elements['rot_2'] = rotations[:, 2]
    elements['rot_3'] = rotations[:, 3]
    
    # Write PLY
    el = PlyElement.describe(elements, 'vertex')
    PlyData([el]).write(output_path)
    
    print(f"✅ Saved Gaussian Splat PLY: {output_path}")
    return output_path

# Convert mesh to Gaussian format
gs_ply_path = f"{OUTPUT_DIR}/gaussian_splat.ply"
mesh_to_gaussian_ply(mesh, gs_ply_path, num_samples=100000)

## 🔟 Download Results

In [ ]:
from google.colab import files

print("📥 Download your 3D models:")
print("\n1. OBJ mesh (with vertex colors):")
files.download(obj_path)

print("\n2. GLB mesh (for web viewers):")
files.download(glb_path)

print("\n3. Gaussian Splat PLY (for gsplat optimization):")
files.download(gs_ply_path)

## 📊 Render Preview Video (Optional)

In [ ]:
from tsr.utils import save_video
from IPython.display import HTML
from base64 import b64encode

# Render 30 views around the object
print("🎬 Rendering preview video...")
with torch.no_grad():
    render_images = model.render(
        scene_codes, 
        n_views=30, 
        return_type="pil"
    )

# Save video
video_path = f"{OUTPUT_DIR}/render.mp4"
save_video(render_images[0], video_path, fps=30)
print(f"✅ Saved video: {video_path}")

# Display in notebook
mp4 = open(video_path, 'rb').read()
data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
HTML(f'<video width=400 controls autoplay loop><source src="{data_url}" type="video/mp4"></video>')

---

## ✅ Next Steps

The `gaussian_splat.ply` file can now be used with:

1. **SyncDreamer** - Generate 16 consistent multi-view images
2. **gsplat optimization** - Refine the Gaussian Splats using multi-view supervision
3. **SDXL Enhancement** - Enhance rendered views with diffusion
4. **MVCRM Refinement** - Back-project enhancements into 3D

Run the **Master Pipeline notebook** to execute the full Glimpse3D workflow!